### Install Packages

In [ ]:
install.packages("BiocManager")
BiocManager::install(c('limma','DESeq2','AnnotationDbi','org.At.tair.db','pathview','gage','gageData','GO.db','GOstats'))
install.packages(c("dplyr","gplots","ggplot2","ggrepel"))

### Load Libraries

In [ ]:
library(DESeq2) 
library(dplyr) 
library(ggplot2)

In [72]:
getwd()

[1] "/home/labuser/MohitPoudel/extra/rna_seq_analysis/scripts"

### Load Count Data and Preprocess:

In [73]:
count_file_path = file.path(dirname(getwd()),"output/counts/merged_counts.csv")
count_file_path

[1] "/home/labuser/MohitPoudel/extra/rna_seq_analysis/output/counts/merged_counts.csv"

In [74]:
countData = read.csv(count_file_path, sep=" ",header=TRUE)

In [75]:
head(countData)

,id,tr721,tr722,con723,con725
,<chr>,<int>,<int>,<int>,<int>
1,FBgn0085804,0,0,0,0
2,FBgn0267431,104,131,77,57
3,FBgn0039987,0,0,0,0
4,FBgn0058182,0,0,0,0
5,FBgn0267430,1045,992,700,729
6,FBgn0266747,132,134,82,126


In [76]:
colnames(countData)

[1] "id"     "tr721"  "tr722"  "con723" "con725"

In [77]:
rownames(countData) = countData$id

In [78]:
countData=countData[,-1]

In [79]:
summary(countData)
# difference is already observed between control and treatment samples

     tr721            tr722            con723             con725        
 Min.   :     0   Min.   :     0   Min.   :     0.0   Min.   :     0.0  
 1st Qu.:     8   1st Qu.:     8   1st Qu.:     6.0   1st Qu.:     8.0  
 Median :   183   Median :   206   Median :   141.0   Median :   176.0  
 Mean   :  2454   Mean   :  2836   Mean   :  2175.1   Mean   :  2110.7  
 3rd Qu.:  1160   3rd Qu.:  1408   3rd Qu.:   852.5   3rd Qu.:   978.5  
 Max.   :728263   Max.   :526754   Max.   :987462.0   Max.   :782962.0  

In [80]:
colSums(countData)

tr721    tr722   con723   con725 
43096987 49796402 38193053 37060965

In [84]:
# Open a png device in the desired directory (relative path)
png("../output/plots/feature_counts.png")

# Set margins
par(mar=c(8,4,4,1) + 0.5)

# Create the barplot
barplot(colSums(countData)/1e6, 
        col="skyblue",
        main="Feature Counts (in Millions)", 
        ylab="Read Counts (Millions)", 
        xlab="Samples",
        cex.axis=1.5,   
        cex.lab=1.5,    
        cex.names=1.5,  
        ylim=c(0, 50))   # Set y-axis limit to 50

# Close the device to finalize the Plot
dev.off()



pdf 
  2

In [85]:
# Histogram to see the value distribution for each samples

## Open a png device in the desired directory (relative path)
png("../output/plots/value_dis.png")

par(mfrow=c(2, 2))

## Create histograms for each sample
hist(countData$tr721, br=10, main="Sample tr721", xlab="Counts", xlim=c(0, 500000))
hist(countData$tr722, br=10, main="Sample tr722", xlab="Counts", xlim=c(0, 500000))
hist(countData$con723, br=10, main="Sample con723", xlab="Counts", xlim=c(0, 500000))
hist(countData$con725, bre=10, main="Sample con725", xlab="Counts", xlim=c(0, 500000))

# Reset the plotting layout to default
par(mfrow=c(1, 1))

# Close the device to finalize the Plot
dev.off()


pdf 
  2

In [13]:
logCount = log2(1 + countData)
# logCount


In [86]:
# Histogram to see the value distribution for each samples

png("../output/plots/log_dis.png")
par(mfrow=c(2, 2))

# Create histograms for each sample
plot(logCount[,1], logCount[,2])
plot(logCount[,1], logCount[,3])
plot(logCount[,1], logCount[,4])
plot(logCount[,2], logCount[,3])

# Reset the plotting layout to default
par(mfrow=c(1, 1))

dev.off()

pdf 
  2

### Loading in DEseq 

In [15]:
sample_type = c("treatment","treatment","control","control")

In [16]:
colData = as.data.frame(cbind(colnames(countData), sample_type))

In [17]:
colData

V1,sample_type
<chr>,<chr>
tr721,treatment
tr722,treatment
con723,control
con725,control


In [18]:
dds = DESeqDataSetFromMatrix(countData, colData, design = ~sample_type)

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”


In [19]:
dds0 = DESeq(dds) # model fitting

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



In [20]:
nrow(dds0)  # no. of genes in the resulting object
dds = dds0[rowSums(counts(dds0)) > 5,] # filter out genes with too-low count

[1] 17559

In [21]:
dim(dds)[1] 

[1] 14725

In [22]:
sizeFactors(dds)
# DESeq2 calculates size factors by using a method that is based on the median of ratios approach, which works as follows:
# For each sample, it computes the ratio of the read count of each gene to the geometric mean of that gene's count across all samples.
# These ratios are then used to calculate a size factor for each sample. This factor is the median of these ratios for each sample, \n 
# ensuring that the gene expression levels are scaled to a common baseline.

tr721     tr722    con723    con725 
1.0718800 1.2790767 0.8210449 0.9035006

## PCA Plot

In [23]:
rld = rlog(dds) # obtaining regularized log counts

In [24]:
pcaData <- plotPCA(rld, intgroup = c("sample_type"), returnData = TRUE)
pcaData

using ntop=500 top features by variance



,PC1,PC2,group,sample_type,name
,<dbl>,<dbl>,<fct>,<fct>,<chr>
tr721,-11.03007,1.1572897,treatment,treatment,tr721
tr722,-20.04683,0.0549765,treatment,treatment,tr722
con723,12.15737,-5.4424311,control,control,con723
con725,18.91953,4.2301649,control,control,con725


In [87]:
# Extract PCA data
pcaData <- plotPCA(rld, intgroup = c("sample_type"), returnData = TRUE)

# Add sample names to the PCA data
pcaData$Sample <- rownames(pcaData)

# Plot PCA with publication-quality style and additional space for points
library(ggplot2)

png("../output/plots/pca.png")

ggplot(pcaData, aes(x = PC1, y = PC2, color = sample_type, label = Sample)) +
  geom_point(size = 4, alpha = 0.7, shape = 16) +  # Larger, semi-transparent points
  geom_text(vjust = -0.5, size = 5, fontface = "italic", color = "black") +  # Larger labels with italic style
  labs(title = "Principal Component Analysis", x = "PC1", y = "PC2") + 
  scale_color_brewer(palette = "Set1") +  # Better color palette for color-blind accessibility
  theme_minimal(base_size = 16) +  # Minimal theme with larger base font size
  theme(
    axis.title = element_text(size = 18, face = "bold"),   # Bold axis titles
    axis.text = element_text(size = 14),    # Larger axis text
    plot.title = element_text(size = 20, face = "bold", hjust = 0.5),  # Bold and centered title
    legend.title = element_text(size = 14, face = "bold"), # Bold legend title
    legend.text = element_text(size = 12),   # Larger legend text
    legend.key.size = unit(1.2, "cm"),  # Adjust legend key size
    plot.margin = margin(1, 1, 1, 2, "cm")  # Increase right margin for extra space
  ) +
  coord_cartesian(clip = "off")  # Allow points outside the default plot area

dev.off()


using ntop=500 top features by variance



pdf 
  2

### Custom Functions:

In [26]:
detectGroups=function(x){
    term=gsub("[0-9]*$","",x)
    # term=gsub("_$","",term)
    # term=gsub("_Rep$","",term)
    return (term)}
# The function returns the category name (control or treatment) from the sample id

In [27]:
detectGroups("con721")


[1] "con"

In [28]:
dist2=function(x){
    as.dist(1-cor(t(x), method="pearson"))}

# calculates the distance matrix for the heatmap based on Pearson correlation

In [29]:
hclust2 = function(x, method="average"){
    hclust(x, method=method)
}
# returns the hierarchical clustering result.


#### Heatmap Code:

In [30]:
library(gplots)


Attaching package: ‘gplots’


The following object is masked from ‘package:IRanges’:

    space


The following object is masked from ‘package:S4Vectors’:

    space


The following object is masked from ‘package:stats’:

    lowess




In [31]:

n = 100 # top 100 genes with the highest variance selected
x = assay(rld) # extracts the expression data from rlog-transformed gene expression data
if(n > dim(x)[1]) n = dim(x)[1] # ensureing n does not exceed the number of rows
x = x[order(apply(x, 1, sd), decreasing = TRUE),] # orders the genes (rows) based on their standard deviation


#### Preprocessing:

In [32]:
x = as.matrix(x[1:n,]) - apply(x[1:n,], 1, mean) 
#  Normalizes the data by centering each gene (row) to have a mean of zero.
cutoff = median(unlist(x)) + 2 * sd(unlist(x)) 
# Calculates a cutoff value to limit extreme values (outliers). 
# This helps in controlling large values that may skew the color scale.
x[x > cutoff] = cutoff # Caps the values greater than the cutoff to the cutoff value.


#### Grouping and Color Mapping:

In [33]:
groups = detectGroups(colnames(x))
groups.colors = rainbow(length(unique(groups)))
# creats color palette with distinct colors for each unique group extracted using detectGroups function

### Heatmap Creation

In [88]:
png("../output/plots/heatmap.png")

lmat = rbind(c(5, 4), c(0, 1), c(3, 2)) # defines layout
lwid = c(1.5, 4)                        # width of plot
lhei = c(1, 0.2, 4)                       # height of plot

heatmap.2(x, distfun = dist2, hclustfun = hclust2,
          col = greenred(75), density.info = "none", trace = "none",
          scale = "none", keysize = 0.2, key = T,
          symkey = F, ColSideColors = groups.colors[as.factor(groups)],
          margins = c(12, 12),              # Increase margins for better label visibility
          cexRow = 1.5,                    # Increase font size of y-axis labels
          cexCol = 1.5,                    # Increase font size of x-axis labels
          srtCol = 45,                     # Rotate x-axis labels
          lmat = lmat, lwid = lwid, lhei = lhei)

dev.off()


pdf 
  2

## DESeq2 SDEG Analysis:

In [35]:
resultsNames(dds)

[1] "Intercept"                        "sample_type_treatment_vs_control"

In [36]:
res=results(dds)
head(res)

log2 fold change (MLE): sample type treatment vs control 
Wald test p-value: sample type treatment vs control 
DataFrame with 6 rows and 6 columns
              baseMean log2FoldChange     lfcSE      stat    pvalue      padj
             <numeric>      <numeric> <numeric> <numeric> <numeric> <numeric>
FBgn0267431   89.07857      0.3484648  0.476756  0.730908 0.4648351  0.805881
FBgn0267430  852.47894      0.0766696  0.248772  0.308192 0.7579362  0.935481
FBgn0266747  116.81037     -0.0726535  0.428796 -0.169436 0.8654538  0.965876
FBgn0086917    1.62885     -1.3409422  3.145707 -0.426277 0.6699061        NA
FBgn0010247 1939.39505      0.5853793  0.266768  2.194341 0.0282109  0.199505
FBgn0086378  675.35212      0.2385825  0.301530  0.791240 0.4288041  0.783478

In [37]:
summary(res)


out of 14725 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)       : 436, 3%
LFC < 0 (down)     : 927, 6.3%
outliers [1]       : 0, 0%
low counts [2]     : 857, 5.8%
(mean count < 4)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



### MA plot

In [89]:
png("../output/plots/ma_plot.png")

res=results(dds, lfcThreshold=0.01)
DESeq2::plotMA(res,ylim=c(-5,5))

dev.off()

pdf 
  2

### Volcano Plot:

In [90]:
png("../output/plots/volcano_plot.png")

# Filter the dataset to remove rows with NA values in log2FoldChange or padj
res1 <- as.data.frame(res) %>%
  filter(!is.na(log2FoldChange) & !is.na(padj)) %>%
  mutate(
    sig = case_when(
      padj < 0.1 & abs(log2FoldChange) >= 1.0 ~ "FDR<0.1 & |Log2FC|≥1",
      TRUE ~ "Not Sig"
    )
  )

# Create the volcano plot
ggplot(res1, aes(x = log2FoldChange, y = -log10(padj))) +
  geom_point(aes(color = sig), size = 2, alpha = 0.8) +  # Use points with transparency
  scale_color_manual(
    values = c("FDR<0.1 & |Log2FC|≥1" = "red", "Not Sig" = "black"),
    name = "Significance"
  ) +
  theme_minimal(base_size = 14) +  # Clean theme with larger font size
  labs(
    title = "Volcano Plot",
    x = expression(Log[2] ~ Fold ~ Change),
    y = expression(-Log[10] ~ Adjusted ~ p ~ Value)
  ) +
  theme(
    plot.title = element_text(hjust = 0.5, face = "bold"),  # Center and bold title
    legend.position = "top",  # Move legend to top
    axis.text = element_text(size = 12),  # Larger axis text size
    axis.title = element_text(size = 14, face = "bold")  # Larger axis titles
  ) +
  geom_hline(yintercept = -log10(0.1), linetype = "dashed", color = "blue", linewidth = 0.7) +  # Significance threshold line
  geom_vline(xintercept = c(-1, 1), linetype = "dashed", color = "blue", linewidth = 0.7)  # Fold-change threshold lines

dev.off()


pdf 
  2

### Normalized Count Plot for the Top Gene

In [91]:
png("../output/plots/top_gene_norm_count_plot.png")

res=res[order(abs(res$log2FoldChange),decreasing=TRUE),]
topGene=rownames(res)[1]
plotCounts(dds, gene=topGene, intgroup=c("sample_type"))

dev.off()

pdf 
  2

## GO ANALYSIS:

In [41]:
library(AnnotationDbi)
BiocManager::install("org.Dm.eg.db") # For Drosophila melanogaster
library(org.Dm.eg.db)  



Attaching package: ‘AnnotationDbi’


The following object is masked from ‘package:dplyr’:

    select


'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.r-project.org

Bioconductor version 3.20 (BiocManager 1.30.25), R 4.4.2 (2024-10-31)

Warning message:
“package(s) not installed when version(s) same as or greater than current; use
  `force = TRUE` to re-install: 'org.Dm.eg.db'”
Old packages: 'BH', 'Biostrings', 'gower', 'openssl', 'parallelly', 'pillar',
  'survival', 'textshaping'





In [42]:
library(GO.db)
library(GOstats)
library(gage)



Loading required package: Category

Loading required package: Matrix


Attaching package: ‘Matrix’


The following object is masked from ‘package:S4Vectors’:

    expand


Loading required package: graph


Attaching package: ‘GOstats’


The following object is masked from ‘package:AnnotationDbi’:

    makeGOGraph




In [43]:
columns(org.Dm.eg.db)

[1] "ACCNUM"       "ALIAS"        "ENSEMBL"      "ENSEMBLPROT"  "ENSEMBLTRANS"
 [6] "ENTREZID"     "ENZYME"       "EVIDENCE"     "EVIDENCEALL"  "FLYBASE"     
[11] "FLYBASECG"    "FLYBASEPROT"  "GENENAME"     "GENETYPE"     "GO"          
[16] "GOALL"        "MAP"          "ONTOLOGY"     "ONTOLOGYALL"  "PATH"        
[21] "PMID"         "REFSEQ"       "SYMBOL"       "UNIPROT"

In [44]:
rownames(head(res))

[1] "FBgn0263407" "FBgn0261399" "FBgn0053300" "FBgn0266070" "FBgn0267663"
[6] "FBgn0035845"

In [45]:
# Map FBgn IDs to Ensembl IDs
res$ENTREZ = mapIds(org.Dm.eg.db,
                     key = rownames(res),  # Assuming rownames(res) are FBgn IDs
                     column = "ENTREZID",   # Specify the Ensembl column
                     keytype = "FLYBASE",  # Specify the keytype as FLYBASE
                     multiVals = "first")  # Handle multiple matches by taking the first match


'select()' returned 1:1 mapping between keys and columns



In [46]:
res$SYMBOL = mapIds(org.Dm.eg.db,
                    key=rownames(res),
                    column="SYMBOL",
                    keytype="FLYBASE",
                    multiVals="first")

'select()' returned 1:1 mapping between keys and columns



In [47]:
head(res)

log2 fold change (MLE): sample type treatment vs control 
Wald test p-value: sample type treatment vs control 
DataFrame with 6 rows and 8 columns
             baseMean log2FoldChange     lfcSE      stat      pvalue
            <numeric>      <numeric> <numeric> <numeric>   <numeric>
FBgn0263407  186.4330      -11.21204   1.50016  -7.47392 7.79388e-14
FBgn0261399  235.7716       11.10903   1.48313   7.49027 6.88198e-14
FBgn0053300  119.7278      -10.57336   1.50571  -7.02219 2.18661e-12
FBgn0266070  107.0638        9.96861   1.51273   6.58982 4.40789e-11
FBgn0267663   43.2547       -9.10615   1.58908  -5.73044 1.00238e-08
FBgn0035845 1283.9707       -8.93338   1.15271  -7.74986 9.22022e-15
                   padj      ENTREZ         SYMBOL
              <numeric> <character>    <character>
FBgn0263407 9.82596e-12    12798501 lncRNA:CR43453
FBgn0261399 8.82015e-12     5740113         Pp1-Y1
FBgn0053300 2.19739e-10     2768931         Muc30E
FBgn0266070 3.82053e-09    19834998 lncRNA:CR4

## SDEG hypergeometric test

In [48]:
res_05=as.data.frame(subset(res,padj<0.05))
head(res_05)

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,ENTREZ,SYMBOL
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
FBgn0263407,186.43302,-11.212043,1.500156,-7.473920,7.793883e-14,9.825961e-12,12798501,lncRNA:CR43453
FBgn0261399,235.77158,11.109027,1.483127,7.490273,6.881975e-14,8.820147e-12,5740113,Pp1-Y1
FBgn0053300,119.72783,-10.573362,1.505707,-7.022189,2.186612e-12,2.197387e-10,2768931,Muc30E
FBgn0266070,107.06385,9.968609,1.512729,6.589820,4.407885e-11,3.820535e-09,19834998,lncRNA:CR44823
FBgn0267663,43.25472,-9.106148,1.589084,-5.730440,1.002376e-08,6.205783e-07,26067322,lncRNA:CR46001
FBgn0035845,1283.97072,-8.933378,1.152714,-7.749864,9.220217e-15,1.331937e-12,38907,CG13675


In [49]:
sig_lfc=1
selectGenesUp = unique(res_05[res_05$log2FoldChange > sig_lfc,"ENTREZ"])
selectGenesDn = unique(res_05[res_05$log2FoldChange < (-sig_lfc),"ENTREZ"])
universeGenes = unique(res_05$ENTREZ)



In [50]:
dim(res_05)[1]
range(res_05$log2FoldChange)


[1] 1081

[1] -11.21204  11.10903

In [51]:
length(selectGenesUp)
length(selectGenesDn)

[1] 152

[1] 679

### GO Biological Process:

In [52]:
upParams = new("GOHyperGParams",
               geneIds = selectGenesUp,
               universeGeneIds = universeGenes,
               annotation = "org.Dm.eg.db",
               ontology = "BP",
               pvalueCutoff = 0.01,
               conditional = FALSE,
               testDirection = "over")

dnParams = new("GOHyperGParams",
               geneIds = selectGenesDn,
               universeGeneIds = universeGenes,
               annotation = "org.Dm.eg.db",
               ontology = "BP",
               pvalueCutoff = 0.01,
               conditional = FALSE,
               testDirection = "over")

upBp = hyperGTest(upParams)
dnBp = hyperGTest(dnParams)

In [53]:
summary(upBp)[1:10,]

,GOBPID,Pvalue,OddsRatio,ExpCount,Count,Size,Term
,<chr>,<dbl>,<dbl>,<dbl>,<int>,<int>,<chr>
1,GO:0090304,0.0001087850,2.872810,13.058405,26,103,nucleic acid metabolic process
2,GO:0043170,0.0002857317,2.284441,28.018519,43,221,macromolecule metabolic process
3,GO:0050794,0.0005117877,2.273786,22.440171,36,177,regulation of cellular process
4,GO:0006139,0.0008023183,2.429820,14.579772,26,115,nucleobase-containing compound metabolic process
5,GO:0051716,0.0008465448,2.649127,10.776353,21,85,cellular response to stimulus
6,GO:0051093,0.0012089516,12.103175,1.014245,5,8,negative regulation of developmental process
7,GO:0048522,0.0012106665,2.843381,8.113960,17,64,positive regulation of cellular process
8,GO:0031323,0.0014026244,2.516681,11.156695,21,88,regulation of cellular metabolic process
9,GO:0006259,0.0017043505,4.946502,2.535613,8,20,DNA metabolic process


In [54]:
summary(dnBp)[1:10,]

,GOBPID,Pvalue,OddsRatio,ExpCount,Count,Size,Term
,<chr>,<dbl>,<dbl>,<dbl>,<int>,<int>,<chr>
1,GO:0045924,8.541596e-05,Inf,9.880342,17,17,regulation of female receptivity
2,GO:0060180,8.541596e-05,Inf,9.880342,17,17,female mating behavior
3,GO:0046692,4.562821e-04,Inf,8.136752,14,14,sperm competition
4,GO:0046008,7.958947e-04,Inf,7.555556,13,13,"regulation of female receptivity, post-mating"
5,GO:0019953,9.709946e-04,1.749826,111.589744,130,192,sexual reproduction
6,GO:0044703,1.938649e-03,11.183206,9.299145,15,16,multi-organism reproductive process
7,GO:0007320,3.186017e-03,10.411168,8.717949,14,15,insemination
8,GO:0007620,3.186017e-03,10.411168,8.717949,14,15,copulation
9,GO:0044706,3.186017e-03,10.411168,8.717949,14,15,multi-multicellular organism process


### GO Cellular Component:

In [55]:
upParams = new("GOHyperGParams",
               geneIds = selectGenesUp,
               universeGeneIds = universeGenes,
               annotation = "org.Dm.eg.db",
               ontology = "CC", # cellular components
               pvalueCutoff = 0.01,
               conditional = FALSE,
               testDirection = "over")

dnParams = new("GOHyperGParams",
               geneIds = selectGenesDn,
               universeGeneIds = universeGenes,
               annotation = "org.Dm.eg.db",
               ontology = "CC",
               pvalueCutoff = 0.01,
               conditional = FALSE,
               testDirection = "over")

upCC = hyperGTest(upParams)
dnCC = hyperGTest(dnParams)

In [56]:
summary(upCC)[1:5,]


,GOCCID,Pvalue,OddsRatio,ExpCount,Count,Size,Term
,<chr>,<dbl>,<dbl>,<dbl>,<int>,<int>,<chr>
1,GO:0032991,0.0001548667,2.703448,14.835905,28,123,protein-containing complex
2,GO:0005634,0.0021556880,2.172414,17.127630,28,142,nucleus
3,GO:0140535,0.0025066520,4.490842,2.653576,8,22,intracellular protein-containing complex
4,GO:1990234,0.0076404521,4.627500,1.929874,6,16,transferase complex
5,GO:0005886,0.0085012825,2.285714,8.805049,16,73,plasma membrane


In [57]:
summary(dnCC)[1:5,]

,GOCCID,Pvalue,OddsRatio,ExpCount,Count,Size,Term
,<chr>,<dbl>,<dbl>,<dbl>,<int>,<int>,<chr>
1,GO:0005576,2.033933e-19,6.031503,111.27069,161,188,extracellular region
2,GO:0005615,3.431644e-18,6.333551,98.24965,144,166,extracellular space
3,GO:0015629,4.083513e-03,6.065432,11.24544,17,19,actin cytoskeleton
NA,NA,NA,NA,NA,NA,NA,NA
NA.1,NA,NA,NA,NA,NA,NA,NA


### GO Molecular Functions

In [58]:
upParams = new("GOHyperGParams",
               geneIds = selectGenesUp,
               universeGeneIds = universeGenes,
               annotation = "org.Dm.eg.db",
               ontology = "MF", # molecular functions
               pvalueCutoff = 0.01,
               conditional = FALSE,
               testDirection = "over")

dnParams = new("GOHyperGParams",
               geneIds = selectGenesDn,
               universeGeneIds = universeGenes,
               annotation = "org.Dm.eg.db",
               ontology = "MF",
               pvalueCutoff = 0.01,
               conditional = FALSE,
               testDirection = "over")

upMF = hyperGTest(upParams)
dnMF = hyperGTest(dnParams)

In [59]:
summary(upMF)[1:5,]
summary(dnMF)[1:5,]

,GOMFID,Pvalue,OddsRatio,ExpCount,Count,Size,Term
,<chr>,<dbl>,<dbl>,<dbl>,<int>,<int>,<chr>
1,GO:0004888,0.0004995426,7.846154,1.6903409,7,14,transmembrane signaling receptor activity
2,GO:0003676,0.0023963043,2.380000,11.5909091,21,96,nucleic acid binding
3,GO:0022836,0.0053023253,5.147679,1.8110795,6,15,gated channel activity
4,GO:0019904,0.0062243461,22.609756,0.4829545,3,4,protein domain specific binding
5,GO:0005488,0.0086905151,1.787013,39.2400568,50,325,binding


,GOMFID,Pvalue,OddsRatio,ExpCount,Count,Size,Term
,<chr>,<dbl>,<dbl>,<dbl>,<int>,<int>,<chr>
1,GO:0030246,7.786161e-06,11.07071,20.67045,32,34,carbohydrate binding
2,GO:0004866,1.414871e-05,Inf,13.37500,22,22,endopeptidase inhibitor activity
3,GO:0030414,1.414871e-05,Inf,13.37500,22,22,peptidase inhibitor activity
4,GO:0048029,1.414871e-05,Inf,13.37500,22,22,monosaccharide binding
5,GO:0061135,1.414871e-05,Inf,13.37500,22,22,endopeptidase regulator activity


In [60]:
summary(upMF)$Term

[1] "transmembrane signaling receptor activity"   
[2] "nucleic acid binding"                        
[3] "gated channel activity"                      
[4] "protein domain specific binding"             
[5] "binding"                                     
[6] "ligand-gated monoatomic ion channel activity"
[7] "ligand-gated channel activity"

In [61]:
summary(dnMF)$Term

[1] "carbohydrate binding"                                                                                 
 [2] "endopeptidase inhibitor activity"                                                                     
 [3] "peptidase inhibitor activity"                                                                         
 [4] "monosaccharide binding"                                                                               
 [5] "endopeptidase regulator activity"                                                                     
 [6] "serine-type endopeptidase inhibitor activity"                                                         
 [7] "peptidase regulator activity"                                                                         
 [8] "procollagen-proline 4-dioxygenase activity"                                                           
 [9] "procollagen-proline dioxygenase activity"                                                             
[10] "peptidyl-proline dioxygenase activity"                                                                
[11] "peptidyl-proline 4-dioxygenase activity"                                                              
[12] "oxidoreductase activity"                                                                              
[13] "vitamin binding"                                                                                      
[14] "carboxylic acid binding"                                                                              
[15] "L-ascorbic acid binding"                                                                              
[16] "organic acid binding"                                                                                 
[17] "oxidoreductase activity, acting on paired donors, with incorporation or reduction of molecular oxygen"
[18] "enzyme inhibitor activity"                                                                            
[19] "molecular function inhibitor activity"                                                                
[20] "enzyme regulator activity"                                                                            
[21] "iron ion binding"                                                                                     
[22] "2-oxoglutarate-dependent dioxygenase activity"                                                        
[23] "dioxygenase activity"                                                                                 
[24] "molecular function regulator activity"                                                                
[25] "carboxylic ester hydrolase activity"

### KEGG ANALYSIS:

In [62]:
upKeggParams = new("KEGGHyperGParams",
                   geneIds=selectGenesUp,
                   universeGeneIds=universeGenes,
                   annotation="org.Dm.eg.db",
                   pvalueCutoff=1,
                   testDirection="over")

dnKeggParams = new("KEGGHyperGParams",
                   geneIds=selectGenesDn,
                   universeGeneIds=universeGenes,
                   annotation="org.Dm.eg.db",
                   pvalueCutoff=1,
                   testDirection="over")

upKEGG=hyperGTest(upKeggParams)
dnKEGG=hyperGTest(dnKeggParams)

In [66]:
summary(upKEGG)[1:5,]

,KEGGID,Pvalue,OddsRatio,ExpCount,Count,Size,Term
,<chr>,<dbl>,<dbl>,<dbl>,<int>,<int>,<chr>
1,00563,0.09558824,Inf,0.09558824,1,1,NA
2,00670,0.09558824,Inf,0.09558824,1,1,NA
3,04140,0.09558824,Inf,0.09558824,1,1,NA
4,03450,0.09558824,Inf,0.09558824,1,1,NA
5,04080,0.18267974,10.16667,0.19117647,1,2,NA


In [68]:
summary(dnKEGG)[1:5,]

,KEGGID,Pvalue,OddsRatio,ExpCount,Count,Size,Term
,<chr>,<dbl>,<dbl>,<dbl>,<int>,<int>,<chr>
1,01100,0.0001214777,3.929825,36.948529,48,67,NA
2,00330,0.0005150782,Inf,6.617647,12,12,NA
3,04142,0.0479516594,Inf,2.757353,5,5,NA
4,00903,0.0585963330,4.022727,6.066176,9,11,NA
5,00480,0.0891495640,Inf,2.205882,4,4,NA
